# Throughput Test Report

## Experiment Environment

### Part 1: Server-based Experiment

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-p8sp{font-size:20px;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-p8sp" colspan="2"><span style="font-weight:bold">Test Platform</span></th>
  </tr>
  <tr>
    <td class="tg-7btt">Ref Test Platform</td>
    <td class="tg-us36">HP ProLiant DL360p Gen8 655651-B21</td>
  </tr>
  <tr>
    <td class="tg-amwm">CPU</td>
    <td class="tg-yw4l">2x "IvyBridge" Intel® Xeon® E5-2630 v2 @ 2.6GHz</td>
  </tr>
  <tr>
    <td class="tg-amwm">Memory</td>
    <td class="tg-yw4l">64GB DDR3</td>
  </tr>
  <tr>
    <td class="tg-amwm">Operating System</td>
    <td class="tg-yw4l">Ubuntu 16.04.5</td>
  </tr>
  <tr>
    <td class="tg-amwm">Test SW</td>
    <td class="tg-yw4l">fio-3.8</td>
  </tr>
  <tr>
    <td class="tg-amwm">HBA</td>
    <td class="tg-yw4l">StarTech USB 3.0 Card PEXUSB3S24<br>Ugreen 20231 USB 3.0 to SATA Converter Adapter<br></td>
  </tr>
  <tr>
    <td class="tg-amwm">PCIe</td>
    <td class="tg-yw4l">Gen 3.0</td>
  </tr>
</table>


<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-p8sp{font-size:20px;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-p8sp" colspan="2"><span style="font-weight:bold">Device Under Test</span></th>
  </tr>
  <tr>
    <td class="tg-7btt">Mfgr</td>
    <td class="tg-us36">Toshiba HG6<br></td>
  </tr>
  <tr>
    <td class="tg-amwm">Model No.</td>
    <td class="tg-yw4l">THNSNJ256G8NU</td>
  </tr>
  <tr>
    <td class="tg-amwm">S/N</td>
    <td class="tg-yw4l">15NS100WT9JW</td>
  </tr>
  <tr>
    <td class="tg-amwm">Firmware Ver.</td>
    <td class="tg-yw4l">JUGB0101</td>
  </tr>
  <tr>
    <td class="tg-amwm">Capacity</td>
    <td class="tg-yw4l">256 GB</td>
  </tr>
  <tr>
    <td class="tg-amwm">Interface</td>
    <td class="tg-yw4l">M.2 B-M<br></td>
  </tr>
  <tr>
    <td class="tg-amwm">NAND Type</td>
    <td class="tg-yw4l">MLC</td>
  </tr>
  <tr>
    <td class="tg-yw4l"><span style="font-weight:bold">Purge Method</span></td>
    <td class="tg-yw4l">None</td>
  </tr>
  <tr>
    <td class="tg-yw4l"><span style="font-weight:bold">Write Cache</span></td>
    <td class="tg-yw4l">Enabled</td>
  </tr>
</table>


<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-p8sp{font-size:20px;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-p8bj{font-weight:bold;border-color:inherit;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-p8sp" colspan="2"><span style="font-weight:bold">Set Up Parameters</span></th>
  </tr>
  <tr>
    <td class="tg-p8bj">Active Range</td>
    <td class="tg-us36">75%<br></td>
  </tr>
  <tr>
    <td class="tg-p8bj">Pre Condition 1</td>
    <td class="tg-us36">SEQ 128KiB W</td>
  </tr>
  <tr>
    <td class="tg-p8bj">    TOIO - TC/QD</td>
    <td class="tg-us36">TC 1 / QD 32</td>
  </tr>
  <tr>
    <td class="tg-p8bj">    Duration</td>
    <td class="tg-us36">2 x User Capacity</td>
  </tr>
  <tr>
    <td class="tg-p8bj">Pre Condition 2</td>
    <td class="tg-us36">SEQ 1024KiB W</td>
  </tr>
  <tr>
    <td class="tg-p8bj">    TOIO - TC/QD</td>
    <td class="tg-us36">TC 1 / QD 32<br></td>
  </tr>
  <tr>
    <td class="tg-p8bj">    Duration</td>
    <td class="tg-us36">2 x User Capacity</td>
  </tr>
  <tr>
    <td class="tg-us36"><span style="font-weight:bold">Purge Method</span></td>
    <td class="tg-us36">None</td>
  </tr>
  <tr>
    <td class="tg-us36"><span style="font-weight:bold">Write Cache</span></td>
    <td class="tg-us36">Enabled</td>
  </tr>
</table>


<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-p8sp{font-size:20px;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-p8bj{font-weight:bold;border-color:inherit;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-p8sp" colspan="2"><span style="font-weight:bold">Test Parameters</span></th>
  </tr>
  <tr>
    <td class="tg-p8bj">Active Range</td>
    <td class="tg-us36">75%</td>
  </tr>
  <tr>
    <td class="tg-p8bj">Test Stimulus 1</td>
    <td class="tg-us36">SEQ</td>
  </tr>
  <tr>
    <td class="tg-p8bj">&nbsp;&nbsp;&nbsp;&nbsp;R/W Mix</td>
    <td class="tg-us36">100:0 / 0:100</td>
  </tr>
  <tr>
    <td class="tg-p8bj">&nbsp;&nbsp;&nbsp;&nbsp;Block Size</td>
    <td class="tg-us36">128KiB / 1024KiB</td>
  </tr>
  <tr>
    <td class="tg-p8bj">    TOIO - TC/QD</td>
    <td class="tg-us36">TC 1 / QD 32</td>
  </tr>
  <tr>
    <td class="tg-p8bj">&nbsp;&nbsp;&nbsp;&nbsp;Steady State</td>
    <td class="tg-us36">1 - 5</td>
  </tr>
</table>